In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, rand
from pyspark.sql.functions import round as ps_round
from pyspark.sql.types import StructField, StringType, StructType

In [ ]:
conf = SparkConf().setAppName("App").setMaster("local[*]")

# Habilitar otimizações e configurações adicionais
conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
conf.set("spark.sql.repl.eagerEval.enabled", "true")
conf.set("spark.sql.repl.eagerEval.truncate", 100)
conf.set("spark.sql.execution.arrow.pyspark.ignore_timezone", "true")

# AWS S3 CONNECTION
AWS_ENDPOINT_URL = "https://s3.bhs.io.cloud.ovh.net"
AWS_ACCESS_KEY = ""
AWS_SECRET_KEY = ""
AWS_REGION = "bhs"

conf.set("spark.jars", "/home/shared/drivers/postgresql-42.7.2.jar")
conf.set("spark.hadoop.fs.s3a.access.key", AWS_ACCESS_KEY)
conf.set("spark.hadoop.fs.s3a.secret.key", AWS_SECRET_KEY)
conf.set("spark.hadoop.fs.s3a.endpoint", AWS_ENDPOINT_URL)
conf.set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
conf.set("spark.jars.packages", "io.delta:delta-spark_2.12:3.1.0,org.apache.hadoop:hadoop-aws:3.2.2")
conf.set("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")

conf.set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
conf.set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")

# Configurações de tempo e legacy
conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")
conf.set("spark.sql.parquet.datetimeRebaseModeInWrite","LEGACY")

# Configurações de memória
conf.set("spark.driver.memory", "60g")
conf.set("spark.executor.memory", "60g")
conf.set("spark.executor.pyspark.memory", "60g")
conf.set("spark.memory.offHeap.enabled", "true")
conf.set("spark.memory.offHeap.size", "60g")

# Inicializa o SparkSession com a configuração
spark = SparkSession.builder.config(conf=conf).getOrCreate()

print("Spark session configurada com sucesso!")

25/05/07 09:50:12 WARN Utils: Your hostname, greca resolves to a loopback address: 127.0.1.1; using 192.168.18.13 instead (on interface enp7s0)
25/05/07 09:50:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/greca/anaconda3/envs/driva_ecomm/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/greca/.ivy2/cache
The jars for the packages stored in: /home/greca/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-49a38ede-1c00-448d-990c-0013866795e2;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
:: resolution report :: resolve 330ms :: artifacts dl 18ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	io.delta#delta-spark_2.12;3.1.0 from central in [default]
	io.delta#delta-storage;3.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	---------------------

Spark session configurada com sucesso!


25/05/07 09:50:17 WARN SparkSession: Cannot use io.delta.sql.DeltaSparkSessionExtension to configure session extensions.
java.lang.NoClassDefFoundError: org/apache/spark/sql/catalyst/analysis/UnresolvedLeafNode
	at java.lang.ClassLoader.defineClass1(Native Method)
	at java.lang.ClassLoader.defineClass(ClassLoader.java:756)
	at java.security.SecureClassLoader.defineClass(SecureClassLoader.java:142)
	at java.net.URLClassLoader.defineClass(URLClassLoader.java:473)
	at java.net.URLClassLoader.access$100(URLClassLoader.java:74)
	at java.net.URLClassLoader$1.run(URLClassLoader.java:369)
	at java.net.URLClassLoader$1.run(URLClassLoader.java:363)
	at java.security.AccessController.doPrivileged(Native Method)
	at java.net.URLClassLoader.findClass(URLClassLoader.java:362)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:418)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:351)
	at java.lang.ClassLoader.defineClass1(Native Method)
	at java.lang.ClassLoader.defineClass(ClassLoader.java:75

In [ ]:
DB_HOST = "driva-db.driva.io"
DB_PORT = 5432
DB_NAME = "postgres"
DB_SITES_SCHEMA = "sites.vinculados"
DB_EB_DRIVA = "empresas_do_brasil.simplificado"
DB_USER = "rafael_greca"
DB_PASSWORD = "59tzyowc#*IGk"

sites_df = spark.read \
    .format("jdbc") \
    .option("url", f"jdbc:postgresql://{DB_HOST}:{DB_PORT}/{DB_NAME}") \
    .option("dbtable", DB_SITES_SCHEMA) \
    .option("user", DB_USER) \
    .option("password", DB_PASSWORD) \
    .option("driver", "org.postgresql.Driver") \
    .load()

eb_df = spark.read \
    .format("jdbc") \
    .option("url", f"jdbc:postgresql://{DB_HOST}:{DB_PORT}/{DB_NAME}") \
    .option("dbtable", DB_EB_DRIVA) \
    .option("user", DB_USER) \
    .option("password", DB_PASSWORD) \
    .option("driver", "org.postgresql.Driver") \
    .load()

In [ ]:
sites_df.printSchema()

In [ ]:
eb_df.printSchema()

In [ ]:
sites_df = sites_df.filter(col("score_vinculo") >= 0.7)
sites_df.count()

In [ ]:
eb_df = eb_df.filter(
    (col("cnae_principal_divisao") == 85) | # Educação
    (col("cnae_principal_divisao") == 86) | # Saúde
    (col("cnae_principal_classe") == 62023) | # Software / Saas
    (col("cnae_principal_classe") == 62031) # Software / Saas
)
eb_df.count()

In [ ]:
merged_df = sites_df.join(eb_df, on="raiz_cnpj", how="inner")
merged_df = merged_df.select(
    "dominio",
    "raiz_cnpj",
    "cnpj",
    "cnae_principal_divisao",
    "cnae_principal_secao",
    "cnae_principal_grupo",
    "cnae_principal_classe",
    "cnae_principal_subclasse",
    "segmento",
)
merged_df.count()

In [ ]:
new_df = spark.createDataFrame(
    merged_df.rdd.takeSample(
        withReplacement=False,
        num=5000,
        seed=42,
    ),
    schema=merged_df.schema,
)

In [ ]:
new_df.count()

In [ ]:
cnae_divisions = [85, 86, 62]
num_samples = 10000
seed = 42

new_df = spark.createDataFrame(
    spark.sparkContext.emptyRDD(),
    schema=merged_df.schema,
)

# field = [
#     StructField("raiz_cnpj", StringType(), True),
#     StructField("raiz_cnpj", StringType(), True),
#     StructField("Segmento iugu", StringType(), True),
# ]
# schema = StructType(field)

# new_segment_column = spark.createDataFrame(
#     spark.sparkContext.emptyRDD(),
#     schema=schema,
# )

for cnae in cnae_divisions:
    if cnae in [85, 86]:
        temp = merged_df.filter(col("cnae_principal_divisao") == cnae)
        temp = spark.createDataFrame(
            temp.rdd.takeSample(
                withReplacement=False,
                num=num_samples,
                seed=seed,
            ),
            schema=merged_df.schema,
        )

        # if cnae == 85:
        #     temp_segment = temp.withColumn("Segmento iugu", lit("Educação")).select("raiz_cnpj", "cnpj", "Segmento iugu")
        # elif cnae == 86:
        #     temp_segment = temp.withColumn("Segmento iugu", lit("Saúde")).select("raiz_cnpj", "cnpj", "Segmento iugu")

    else:
        temp = merged_df.filter(
            (col("cnae_principal_classe") == 62023) |
            (col("cnae_principal_classe") == 62031)
        )
        
        temp = spark.createDataFrame(
            temp.rdd.takeSample(
                withReplacement=False,
                num=num_samples,
                seed=seed,
            ),
            schema=merged_df.schema,
        )
        # temp_segment = temp.withColumn("Segmento iugu", lit("Saas")).select("raiz_cnpj", "cnpj", "Segmento iugu")
        
    new_df = new_df.union(temp)
    # new_segment_column = new_segment_column.union(temp_segment)

# new_df = new_df.join(new_segment_column, on="raiz_cnpj", how="inner")
new_df.printSchema()

In [ ]:
new_df.count()

In [ ]:
# new_df.write.save("s3a://drivalake/raw/sites/iugu/iugu_chatgpt.parquet")

In [ ]:
spark.stop()

In [ ]:
import pandas as pd

df = pd.read_parquet("../data/new_iugu_saas2_with_html.parquet", engine="pyarrow")
df.head()

In [ ]:
label = pd.read_parquet("../data/iugu_with_saas2.parquet", engine="pyarrow")
label = label.rename(columns={"dominio": "host"})
label

In [ ]:
df = df.merge(label, how="inner", on="host")
df = df[
    [
        "url",
        "host",
        "html",
        "raiz_cnpj",
        "cnpj",
        "cnae_principal_divisao",
        "cnae_principal_secao",
        "cnae_principal_grupo",
        "cnae_principal_classe",
        "cnae_principal_subclasse",
        "segmento",
    ]
]
df.head()

In [ ]:
mapping = {
    85: "Educação",
    86: "Saúde",
    62: "Saas"
}

df["Segmento iugu"] = df["cnae_principal_divisao"].apply(lambda x: mapping[x])
df

In [ ]:
df["Segmento iugu"].value_counts()

In [ ]:
df.to_parquet("../data/new_iugu_saas2_with_html.parquet", engine="pyarrow")